Install the correct version of the required packages (in case you haven't done it yet)

In [ ]:
%pip install -r '../requirements.txt'

### Import libraries

In [1]:
import pandas as pd
import numpy as np 
import tqdm
import joblib
import yaml

# Preprocessing
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,RobustScaler


# Models
from sklearn.ensemble import (
    RandomForestClassifier,
    VotingClassifier
)
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import SVC
from xgboost import XGBClassifier

# Parameter tuning, splitting
from sklearn.model_selection import (
    GridSearchCV,
    RepeatedStratifiedKFold,
    cross_val_score,
    train_test_split,
)

# Metrics
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


### Read dataset

In [ ]:
df = pd.read_csv("../data/dataset_for_training.csv", header=0)

### Get models

In [ ]:
def get_classification_models():
    models = dict()
    models["KNN"] = KNeighborsClassifier()
    models["LOR"] = LogisticRegression()
    models["SVM"] = SVC(probability=True)
    models["RF"] = RandomForestClassifier()
    models["XGB"] = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    return models

### Define pipeline

In [ ]:
def create_pipeline(model_abbreviation, model, categorical_indices, continuous_indices):
    scaler = RobustScaler()
    imputer = SimpleImputer(strategy="median")
    scalingsteps = [("cont_imputer", imputer), ("continuous", scaler)]

    sampler = SMOTE(random_state=42)


    scaling_pipeline = Pipeline(steps=scalingsteps)
    encoding_pipeline = Pipeline(
                steps=[
                    (
                        "categorical",
                        OneHotEncoder(handle_unknown="ignore", drop="if_binary"),
                    )
                ]
            )
    if model_abbreviation == "ensemble":
        finalmodel = model
    else:
        finalmodel = CalibratedClassifierCV(
            base_estimator=model, cv=5, ensemble=True
        )
    preprocess = ColumnTransformer(
            transformers=[
                ("scal", scaling_pipeline, continuous_indices),
                ("cat", encoding_pipeline, categorical_indices),
            ],
            remainder="passthrough",
        )
    pipeline = Pipeline(
            steps=[
                ("preprocess", preprocess),
                ("sampler", sampler),
                ("model", finalmodel),
            ])
    return pipeline

In [ ]:
 def gridsearch(pipeline, abbrev, outcome, X_train, y_train):
    """Gridsearch to find optimal parameters for a ML algorithm based on ROC AUC score.
    Saves:
    Yaml file with the optimal parameters in the model's outcomefolder."""
    file = "../parameters/gridsearch.yaml"
    dict = yaml.safe_load(open(file, "rb"))
    
    if abbrev == "RF":
        param_grid = dict["rf_param"]
        param_grid["model"] = [RandomForestClassifier()]
    elif abbrev == "SVM":
        param_grid = dict["svm_param"]
        param_grid["model"] = [SVC()]
    elif abbrev == "KNN":
        param_grid = dict["knn_param"]
        param_grid["model"] = [KNeighborsClassifier()]
    elif abbrev == "LOR":
        param_grid = dict["lor_param"]
        param_grid["model"] = [LogisticRegression()]
    else:
        param_grid = dict["xgb_param"]
        param_grid["model"] = [XGBClassifier()]
    grid = GridSearchCV(
        pipeline, param_grid, scoring='roc_auc', cv=5, verbose=1, n_jobs=-1
    )
    grid.fit(X_train, y_train)
    grid_params = grid.best_params_
    outputpath = os.path.join(f"../parameters/{outcome}/{abbrev}")
    os.makedirs(outputpath, exist_ok=True)
    with open(os.path.join(outputpath, "bestparameters.yaml"), "w") as outfile:
        yaml.dump(grid_params, outfile, default_flow_style=False)


### Get best parameters

In [ ]:
models = ["LOR", "KNN", "SVM", "RF", "XGB"]
classificationtype = "binary"

In [ ]:
for model_abbreviation in models:
    modeldict = get_classification_models()
    y = df["y"]
    X = df.drop(["y"], axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,stratify=y)
    categorical_features = list(X.select_dtypes(include=["object"]))
    categorical_indices = [X.columns.get_loc(cat) for cat in categorical_features]
    continuous_features = list(X.select_dtypes(include=["float", "int"]))
    continuous_indices = [X.columns.get_loc(cont) for cont in continuous_features]
    pipeline = create_pipeline(model_abbreviation=model_abbreviation, model=modeldict[model_abbreviation], categorical_indices=categorical_indices, continuous_indices=continuous_indices)
    gridsearch(pipeline, model_abbreviation, "M_6weeks", X_train, y_train)

In [ ]:
def get_classification_models_with_parameters(outcome):
    modeldict = dict()
    with open(f'../parameters/{outcome}/KNN/bestparameters.yaml') as knnfile:
        knn = yaml.load(knnfile, Loader=yaml.Loader)
    with open(f'../parameters/{outcome}/LOR/bestparameters.yaml') as lorfile:
        lor = yaml.load(lorfile, Loader=yaml.Loader)
    with open(f'../parameters/{outcome}/SVM/bestparameters.yaml') as svmfile:
        svm = yaml.load(svmfile, Loader=yaml.Loader)
    with open(f'../parameters/{outcome}/RF/bestparameters.yaml') as rffile:
        rf = yaml.load(rffile, Loader=yaml.Loader)
    with open(f'../parameters/{outcome}/XGB/bestparameters.yaml') as xgbfile:
        xgb = yaml.load(xgbfile, Loader=yaml.Loader)

    modeldict["KNN"] = KNeighborsClassifier(
            n_neighbors=knn["model__n_neighbors"],
            weights=knn["model__weights"],
            metric=knn["model__metric"],
        )
    modeldict["LOR"] = LogisticRegression(
            solver=lor["model__solver"],
            C=lor["model__C"],
            max_iter=lor["model__max_iter"],
            penalty=lor["model__penalty"],
            random_state=42,
        )
    modeldict["SVM"] = SVC(
            C=svm["model__C"],
            gamma=svm["model__gamma"],
            kernel="rbf",
            probability=True,
            random_state=42,
        )
    modeldict["RF"] = RandomForestClassifier(
            min_samples_split=rf["model__min_samples_split"],
            n_estimators=rf["model__n_estimators"],
            max_depth=rf["model__max_depth"],
            max_features="sqrt",
            min_samples_leaf=rf["model__min_samples_leaf"],
            random_state=42,
        )
    modeldict["XGB"] = XGBClassifier(
            learning_rate=xgb["model__learning_rate"],
            max_depth=xgb["model__max_depth"],
            n_estimators=xgb["model__n_estimators"],
            min_child_weight=xgb["model__min_child_weight"],
            subsample=xgb["model__subsample"],
            eval_metric=xgb["model__eval_metric"],
            random_state=42,
        )
    return modeldict

In [ ]:
def train(pipeline, X_train, y_train, classificationtype,features):
    pipeline.fit(X_train, np.ravel(y_train))
    y_hat_train = pipeline.predict(X_train)
    y_prob_train = pipeline.predict_proba(X_train)
    train_acc = accuracy_score(y_train, y_hat_train)
    if classificationtype == "binary":
        train_auc = roc_auc_score(y_train, y_prob_train[:, 1])
    else:
        train_auc = roc_auc_score(
            y_train, y_prob_train, multi_class="ovo", average="macro"
        )
    report = classification_report(y_train, y_hat_train)
    print("Train results of {} with {} features".format(pipeline.named_steps["model"], len(features)))
    print("Train accuracy: {:.2f}".format(train_acc))
    print("Train AUC: {:.2f}".format(train_auc))
    print(report)
    return {"train_acc": train_acc, "train_auc": train_auc, "train_report": report}

In [ ]:
def test(pipeline, X_train, y_train, X_test, y_test, classificationtype,features):
    pipeline.fit(X_train, y_train)
    y_hat = pipeline.predict(X_test)
    y_prob = pipeline.predict_proba(X_test)
    test_acc = accuracy_score(y_test, y_hat)
    if classificationtype == "binary":
        test_auc = roc_auc_score(y_test, y_prob[:, 1], average="macro")
        tn, fp, fn, tp = confusion_matrix(y_test, y_hat).ravel()
    else:
        test_auc = roc_auc_score(
            y_test, y_prob, multi_class="ovr", average="macro"
        )
        tn, fp, fn, tp = "nvt", "nvt", "nvt", "nvt"
    F1 = f1_score(y_test, y_hat, average="macro")
    precision = precision_score(y_test, y_hat, average="macro")
    recall = recall_score(y_test, y_hat, average="macro")
    print("Test results of {} with {} features".format(pipeline.named_steps["model"], len(features)))
    print("Test accuracy: {:.2f}".format(test_acc))
    print("Test AUC: {:.2f}".format(test_auc))
    print("Test F1-score: {:.2f}".format(F1))
    report = classification_report(y_test, y_hat)
    print(report)
    return {
        "test_acc": test_acc,
        "test_auc": test_auc,
        "test_report": report,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "tp": tp,
        "test_F1": F1,
        "test_recall": recall,
        "test_precision": precision,
    }

### Create ensemble

In [ ]:
def create_classification_ensemble_model(estimators, X_train, y_train, X_test, y_test, classificationtype,features, categorical_indices, continuous_indices):
    base_estimators = list()  # base models
    weights = list()
    results = dict()
    counter = 0
    models = get_classification_models_with_parameters()
    for abbreviation in estimators:
        ML_model = models[abbreviation]
        pipeline = create_pipeline(abbreviation,ML_model, categorical_indices, continuous_indices)
        train_results = train(pipeline, X_train, y_train, classificationtype,features)
        test_results = test(pipeline, X_train, y_train, X_test, y_test, classificationtype,features)
        weights.append(test_results["test_auc"] * test_results["test_auc"])
        results[abbreviation] = {
            "ACC": test_results["test_acc"],
            "AUC": test_results["test_auc"],
            "RECALL": test_results["test_recall"],
            "PRECISION": test_results["test_precision"],
            "F1": test_results["test_F1"],
        }
        base_estimators.append(
            (
                f"{abbreviation}_{counter}",
                Pipeline(
                    steps=[
                        (
                            f"{abbreviation}_{counter}",
                            pipeline.named_steps["model"],
                        )
                    ]
                ),
            )
        )
        counter += 1
    normalized_weights = [float(i / sum(weights)) for i in weights]
    ensemble = VotingClassifier(
        base_estimators, voting="soft", weights=normalized_weights
    )
    return ensemble, results

### Confidence interval

In [ ]:
def confidence_interval_classification(all_accuracy, all_aucs, all_recall, all_precision, all_f1):
    all_accuracy.sort()
    all_aucs.sort()
    all_recall.sort()
    all_precision.sort()
    all_f1.sort()

    print(f"Confidence interval accuracy: {np.array(all_accuracy).mean()} [{np.array(all_accuracy).mean() - (2*np.array(all_accuracy).std())}-{np.array(all_accuracy).mean()+(2*np.array(all_accuracy).std())}]\n")
    print(f"Confidence interval AUC: {np.array(all_aucs).mean()} [{np.array(all_aucs).mean() - (2*np.array(all_aucs).std())}-{np.array(all_aucs).mean()+(2*np.array(all_aucs).std())}]\n")
    print(f"Confidence interval recall: {np.array(all_recall).mean()} [{np.array(all_recall).mean() - (2*np.array(all_recall).std())}-{np.array(all_recall).mean()+(2*np.array(all_recall).std())}]\n")
    print(f"Confidence interval precision: {np.array(all_precision).mean()} [{np.array(all_precision).mean() - (2*np.array(all_precision).std())}-{np.array(all_precision).mean()+(2*np.array(all_precision).std())}]\n")
    print(f"Confidence interval F1-score: {np.array(all_f1).mean()} [{np.array(all_f1).mean() - (2*np.array(all_f1).std())}-{np.array(all_f1).mean()+(2*np.array(all_f1).std())}]\n")

### Save results

In [ ]:
def save_pipeline(pipeline, model_abbreviation, features, train_results, test_results, split):
    joblib.dump(pipeline, f"../results/{model_abbreviation}_bestpipeline.pkl")
    with open(f'../results/{model_abbreviation}_bestpipeline_results.txt', 'w') as file:
        file.write("Data was split with random_state={}\n".format(split))
        file.write("Train accuracy: {:.2f}\n".format(train_results["train_acc"]))
        file.write("Train AUC of {:.2f}\n".format(train_results["train_auc"]))
        file.write("{}\n".format(train_results["train_report"]))
        file.write("Test accuracy: {:.2f}\n".format(test_results["test_acc"]))
        file.write("Test AUC of {:.2f}\n".format(test_results["test_auc"]))
        file.write("TN: {}\tFP: {}\tFN: {}\tTP: {}\n".format(test_results["tn"], test_results["fp"], test_results["fn"], test_results["tp"]))
        file.write("{}\n\n".format(test_results["test_report"]))
        file.write("Features:\n")
        for feat in features:
            file.write(f"{feat}\n")

### Train loop

In [ ]:
def train_test_loop(dataset, model_abbreviation, outcome="M_6weeks", loops=1, save=False):
    best_auc = 0
    all_aucs = []
    all_recall = []
    all_precision = []
    all_f1 = []
    all_accuracies = []
    for i in tqdm.tqdm(range(0,int(loops))):
        y = dataset["y"]
        X = dataset.drop(["y"], axis=1)
        categorical_features = list(X.select_dtypes(include=["object"]))
        categorical_indices = [X.columns.get_loc(cat) for cat in categorical_features]
        continuous_features = list(X.select_dtypes(include=["float", "int"]))
        continuous_indices = [X.columns.get_loc(cont) for cont in continuous_features]
        features = X.columns
        # Split dataset
        X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20,stratify=y,random_state=i)
        print("Training set: {}, Test set: {}".format(len(X_train), len(X_test)))

        if model_abbreviation == "ensemble":
            voting_classifier, results_seperate_estimators = create_classification_ensemble_model(models, X_train, y_train, X_test, y_test, classificationtype,features,  categorical_indices, continuous_indices)
            pipeline = create_pipeline(model_abbreviation, voting_classifier, categorical_indices, continuous_indices)
           
        else:
            modeldict = get_classification_models_with_parameters(outcome)
            model = modeldict[model_abbreviation]
            pipeline = create_pipeline(model_abbreviation, model, categorical_indices, continuous_indices)
        train_results = train(pipeline, X_train, y_train, classificationtype,features)
        test_results = test(pipeline, X_train, y_train, X_test, y_test, classificationtype,features)
        all_accuracies.append(test_results["test_acc"])
        all_aucs.append(test_results["test_auc"])
        all_recall.append(test_results["test_recall"])
        all_precision.append(test_results["test_precision"])
        all_f1.append(test_results["test_F1"])
        
        if save:
            if test_results["test_auc"] > best_auc:
                save_pipeline(pipeline, model_abbreviation, features, train_results, test_results, i)
                best_auc = test_results["test_auc"]
    if loops > 20:
        confidence_interval_classification(all_accuracies,all_aucs,all_recall,all_precision,all_f1)

In [ ]:
train_test_loop(df, "RF", "M_6weeks")

In [ ]:
train_test_loop(df, "ensemble", loops=20, save=True)